# Author Identification

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import zipfile
with zipfile.ZipFile('/kaggle/input/spooky-author-identification/train.zip', 'r') as zip_ref:
    zip_ref.extractall('./')
    
with zipfile.ZipFile('/kaggle/input/spooky-author-identification/test.zip', 'r') as zip_ref:
    zip_ref.extractall('./')

This is created as a part of the [kaggle competition](https://www.kaggle.com/c/spooky-author-identification/data?select=train.zip) to identify the author based on several phrases of the books they have written. The given data set contains two training and test csv files, where I have generated a validation set from the training data set.
Here, I have used the **nltk** toolkit in python to conduct the natural language processing, and created several classification models to find the best suited model to identify the author. 

In [ ]:
import pandas as pd
import numpy as np

In [ ]:
df_train = pd.read_csv('train.csv',index_col=0)
df_test = pd.read_csv('test.csv',index_col=0)

print(df_train.shape)
print(df_test.shape)
df_train.head()


In [ ]:
lables = df_train.author.unique().tolist()
lables

Create function to clean the text data. This function removes the punctuations, makes all the text in to lower case, words are seperated by non word charachters, stopwords are removed and then stemmed. 

In [ ]:
import nltk
import re
import string

stopwords = nltk.corpus.stopwords.words('english')
ps = nltk.PorterStemmer()

def clean_data(text) : 
    text = "".join([word.lower() for word in text if word not in string.punctuation])
    tokens = re.split('\W+',text)
    text = [ps.stem(word) for word in tokens if word not in stopwords]
    return text


The training data set obtained form the given csv is seperated to a training and a validation set.  
To vectorize, **TFIDF vectorization** is used since it gives higher weights to unique words.   

In [ ]:
from sklearn.model_selection import train_test_split
x_train, x_valid, y_train, y_valid = train_test_split(df_train['text'],df_train['author'],test_size = 0.3, random_state=42)
x_test = df_test['text']

from sklearn.feature_extraction.text import TfidfVectorizer

tfidf_vect = TfidfVectorizer(analyzer = clean_data)
tfidf_vect_fit = tfidf_vect.fit(x_train)

tfidf_train = tfidf_vect_fit.transform(x_train)
tfidf_valid  = tfidf_vect_fit.transform(x_valid)
tfidf_test  = tfidf_vect_fit.transform(x_test)

x_train_vect = pd.DataFrame(tfidf_train.toarray())
x_valid_vect = pd.DataFrame(tfidf_valid.toarray())
x_test_vect  = pd.DataFrame(tfidf_test.toarray())


In [ ]:
x_train_vect.head()

In [ ]:
x_valid_vect.head()

In [ ]:
x_test_vect.head()

### Feature Extraction

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline

Drop low value words and keep rare words  

In [ ]:
# col_list = x_train_vect[x_train_vect.max(axis=0)>0.1].index.values.tolist()
# print(len(col_list))

# x_train_vect = x_train_vect[col_list]
# x_test_vect = x_test_vect[col_list]
# x_train_vect.head()

### Evaluate models

Here, we analyze several types of Classification models, and find the accuray of each model and visualize them by confusion metrices. To find the best parameters, GridSearchCV is run on each model.

In [ ]:
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import MultinomialNB
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import precision_recall_fscore_support as score
from sklearn.metrics import plot_confusion_matrix


#### Naive Bayes

In [ ]:
nb = MultinomialNB()
param = {'alpha':[0.01, 0.1, 1,10, 100]}

gsn = GridSearchCV(nb, param, cv=5) 
gb_fit = gsn.fit(x_train_vect,y_train)
pd.DataFrame(gb_fit.cv_results_).sort_values('mean_test_score',ascending=False)[0:3]

In [ ]:
y_pred_nb = gsn.best_estimator_.predict(x_valid_vect)
precision,recall,fscore,support = score(y_valid,y_pred_nb, average='weighted')
print('precision:{}, recall:{}, accuracy:{}'.format(round(precision,3), round(recall,3), round((y_valid==y_pred_nb).sum()/len(y_pred_nb) ,3)))

In [ ]:
plot_confusion_matrix(gsn.best_estimator_, x_valid_vect, y_valid, normalize='pred',xticks_rotation ='vertical')

#### Logistic Regression

In [ ]:
lr = LogisticRegression()
param = {'C' : [0.01,0.1,1,10,100], 'max_iter': [100000]}

gsl = GridSearchCV(lr, param, cv=5) 
lr_fit = gsl.fit(x_train_vect,y_train)
pd.DataFrame(lr_fit.cv_results_).sort_values('mean_test_score',ascending=False)[0:3]

In [ ]:
y_pred_lr = gsl.best_estimator_.predict(x_valid_vect)
precision,recall,fscore,support = score(y_valid,y_pred_lr, average='weighted')
print('precision:{}, recall:{}, accuracy:{}'.format(round(precision,3), round(recall,3), round((y_valid==y_pred_lr).sum()/len(y_pred_lr) ,3)))

In [ ]:
plot_confusion_matrix(gsl.best_estimator_, x_valid_vect, y_valid, normalize='pred',xticks_rotation ='vertical')

#### Random Forest Classifier

In [ ]:
rf = RandomForestClassifier()
param = {'n_estimators': [100,200],'max_depth' : [None] , 'n_jobs' : [-1]}

gs = GridSearchCV(rf, param, cv=3)
rf_fit = gs.fit(x_train_vect,y_train)
pd.DataFrame(rf_fit.cv_results_).sort_values('mean_test_score',ascending=False)

In [ ]:
y_pred_rf = gs.best_estimator_.predict(x_valid_vect)
precision,recall,fscore,support = score(y_valid,y_pred_rf, average='weighted')
print('precision:{}, recall:{}, accuracy:{}'.format(round(precision,3), round(recall,3), round((y_valid==y_pred_rf).sum()/len(y_pred_rf) ,3)))

In [ ]:
plot_confusion_matrix(gs.best_estimator_, x_valid_vect, y_valid, normalize='pred', xticks_rotation ='vertical' )

#### Gradient Boosing

In [ ]:
# gb = GradientBoostingClassifier()
# param = {'n_estimators':[50,100,200], 'max_depth': [10,20],'learning_rate':[0.1], 'n_jobs' : [-1]}

# gsg = GridSearchCV(gb, param, cv=5) 
# gb_fit = gsg.fit(x_train_vect,y_train)
# pd.DataFrame(gb_fit.cv_results_).sort_values('mean_test_score',ascending=False)[0:3]

In [ ]:
# y_pred_gb = gsg.best_estimator_.predict(x_valid_vect)
# precision,recall,fscore,support = score(y_valid,y_pred_gb, average='weighted')
# print('precision:{}, recall:{}, accuracy:{}'.format(round(precision,3), round(recall,3), round((y_valid==y_pred_gb).sum()/len(y_pred_gb) ,3)))

In [ ]:
# plot_confusion_matrix(gsg.best_estimator_, x_valid_vect, y_valid, normalize='pred',xticks_rotation ='vertical')

### Create the submision file based on the best model

the model that perfomed the best was the Naive Bayes model. Hence, the submission file was created with the probabilities for each category for the test.csv

In [ ]:
y_test = gsn.best_estimator_.predict_proba(x_test_vect)

In [ ]:
y_test_out = pd.DataFrame(y_test,columns=[['EAP','HPL','MWS']],index=x_test.index)

In [ ]:
y_test_out.to_csv('submission.csv')